# GW Releases

In [1]:
import requests
import pandas as pd
import re
import json
from IPython.core.display import display, HTML
from modules.Network import *
display(HTML("<style>.container { width:70% !important; }</style>"))

In [2]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

We get the json file with the information from https://wmwaredata.s3.us-east-2.amazonaws.com/gw_releases.json

In [3]:
url = 'https://wmwaredata.s3.us-east-2.amazonaws.com/gw_releases.json'
r = requests.get(url, allow_redirects=True)
open('data/gw_releases.json', 'wb').write(r.content)

6914

In [4]:
df = pd.read_json('data/gw_releases.json')
df = df.rename(columns={'sub_repo_commit_url': 'sub_repo_url'})

In [5]:
df

,commit_url,hash,hash_short,release_date,release_name,release_tag,repo_name,repo_url,sub_commit_url,sub_hash,sub_hash_short,sub_name,sub_repo_url
0,https://github.com/k8-proxy/GW-proxy/tree/d8b3...,d8b3bb3db39c7114f2a58b582746cb51ea653b0b,d8b3bb3,2021-02-22T13:57:35Z,,v0.5.2,GW-proxy,https://github.com/k8-proxy/GW-proxy,None,None,None,None,None
1,https://github.com/k8-proxy/s-k8-proxy-rebuild...,ffbcabeaba991606b3e86d9f44f08fe4e2f4a26d,ffbcabe,2021-01-20T09:34:25Z,,v0.1.1,s-k8-proxy-rebuild,https://github.com/k8-proxy/s-k8-proxy-rebuild,None,None,None,None,None
2,https://github.com/k8-proxy/aws-jmeter-test-en...,07e073d2c7781d2073ef42199903507d7c60ece2,07e073d,2021-02-19T11:39:43Z,- Simplified instructions that can help setup ...,v0.3,aws-jmeter-test-engine,https://github.com/k8-proxy/aws-jmeter-test-en...,None,None,None,None,None
3,https://github.com/k8-proxy/icap-infrastructur...,9027b550a263af153220a3a98377b8e34cf4e7de,9027b55,2021-02-15T12:18:08Z,,v1.0.1,icap-infrastructure,https://github.com/k8-proxy/icap-infrastructure,None,None,None,None,None
4,https://github.com/k8-proxy/gp-load-balancer/t...,82c3df98dc137c3b7095d3041d9ba57c3b8a2c6d,82c3df9,2021-02-15T11:04:31Z,,v0.1.1,gp-load-balancer,https://github.com/k8-proxy/gp-load-balancer,None,None,None,None,None
5,https://github.com/k8-proxy/k8-rebuild/tree/fc...,fc542782e003b292cd5951651ba5997003f1ecad,fc54278,2021-02-22T14:27:56Z,,sow-v0.1,k8-rebuild,https://github.com/k8-proxy/k8-rebuild,https://github.com/k8-proxy/k8-rebuild-file-dr...,79b14699988edbe0fb0c64a3b2c4d4c0eb60587e,79b1469,k8-rebuild-file-drop,https://github.com/k8-proxy/k8-rebuild-file-drop
6,https://github.com/k8-proxy/k8-rebuild/tree/fc...,fc542782e003b292cd5951651ba5997003f1ecad,fc54278,2021-02-22T14:27:56Z,,sow-v0.1,k8-rebuild,https://github.com/k8-proxy/k8-rebuild,https://github.com/k8-proxy/k8-rebuild-rest-ap...,34a3f777159ff566102850064d2017a6054b7591,34a3f77,k8-rebuild-rest-api,https://github.com/k8-proxy/k8-rebuild-rest-api
7,https://github.com/k8-proxy/k8-rebuild-folder-...,11be72fb5adf41d53be98fdee083446a7938b622,11be72f,2021-02-22T15:30:13Z,,v0.1.2,k8-rebuild-folder-to-folder,https://github.com/k8-proxy/k8-rebuild-folder-...,None,None,None,None,None
8,https://github.com/k8-proxy/vmware-scripts/tre...,6fd4bebf6ab2eaae1a21f09b4e42ca747d0f97fd,6fd4beb,2021-02-15T11:05:13Z,,v0.1.6,vmware-scripts,https://github.com/k8-proxy/vmware-scripts,None,None,None,None,None
9,https://github.com/k8-proxy/icap-client/tree/a...,a112f474f1b3ebbddf15700c784dd28bcbefcf4e,a112f47,2021-02-17T19:33:50Z,,v0.2.1,icap-client,https://github.com/k8-proxy/icap-client,https://github.com/k8-proxy/go-icap-client/blo...,d095450c1e13fd7d871bea7e55727c534918b4ad,d095450,go-icap-client,https://github.com/k8-proxy/go-icap-client


In [6]:
# CREATE JSON FILE WITH NODES AND EDGES

network = Network()
network.groups = ['GWReleases', 'Repo', 'SubRepo']
network.groupSettings = group_settings_GW

# GW-Releases node
network.add_node(label='GW-Releases', group='GWReleases')

for repo in set(list(df['repo_url'])):
    # REPO
    repo_name = re.sub('https://github.com/', '', repo)
    #hash_ = df[df['repo_url']==repo]['hash'].iloc[0]
    hash_short = df[df['repo_url']==repo]['hash_short'].iloc[0]
    tag = df[df['repo_url']==repo]['release_tag'].iloc[0]
    label = repo_name + '\n' + tag + '\n' + hash_short
    network.add_node(label, group='Repo')
    network.add_edge(network.nodesIds['GW-Releases'][0], network.nodesIds[label][0])
    
    # SUB REPO
    df1 = df[df['repo_url'] == repo].dropna().reset_index()
    for i in range(len(df1)):
        sub_repo = df1['sub_repo_url'].iloc[i]
        sub_repo_name = re.sub('https://github.com/', '', sub_repo)
        #sub_hash = df1[df1['sub_repo_commit_url']==sub_repo]['sub_hash'].iloc[0]
        sub_hash_short = df1[df1['sub_repo_url']==sub_repo]['sub_hash_short'].iloc[0]
        label2 = sub_repo_name + '\n' + sub_hash_short
        network.add_node(label2, group='SubRepo')
        network.add_edge(network.nodesIds[label][0], network.nodesIds[label2][0])
        
network.save_to_json('data/gw_releases_nodes_edges.json')

In [7]:
%%html
<div id="mynetwork0"></div>

In [8]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});


require(['vis'], function(vis){
    
    var json = $.getJSON("data/gw_releases_nodes_edges.json")
      .done(function(data){
        var data = {
          nodes: data.nodes,
          edges: data.edges
        };
        var network = new vis.Network(container, data, options);
      });
    
    var options = {
      width: '1200px',
      height: '900px',
      locale: 'en',
      physics: false,
      interaction: {
        hover:true, 
        tooltipDelay: 300
      },        
      layout: {
        randomSeed: 1,
        improvedLayout: true,
        hierarchical: {
          enabled: true,
          direction: 'LR',
          nodeSpacing: 200,
          levelSeparation: 250,  
          sortMethod: 'directed',
        },
      },
    }; 
    
    var container = document.getElementById("mynetwork0");
    
});

<IPython.core.display.Javascript object>

In [9]:
%%html
<div id="mynetwork1"></div>

In [10]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});

require(['vis'], function(vis){ 
    
    var json = $.getJSON("data/gw_releases_nodes_edges.json")
      .done(function(data){
        var data = {
          nodes: data.nodes,
          edges: data.edges
        };
        var network = new vis.Network(container, data, options);
      });
       
    var options = {      
      width: '1200px',
      height: '1000px',
      physics: true,
    };    
    
    var container = document.getElementById("mynetwork1");
});

<IPython.core.display.Javascript object>